# Predicting Covid-19 Misinformation on Twitter

Creating a predictive model to classify misinformation is a first step in mitigating the spread of misinformation. Today with the ease to spread both information, as well as misinformation, it is necessary to try and stop the spread of misinformation sooner rather than later to prevent its rampant spread and harmful impacts. Particularly with regard to the Covid-19 pandemic, circulating misinformation could have a potential larger-scale impact on people's view of the virus and global health more broadly. Creating a classifier is an automated way to begin identifying and stopping the spread of misinformation, and hopefully, mitigtating the outspread of the Covid-19 pandemic.

## Import Data

In [10]:
#Source:Fighting an Infodemic: COVID-19 Fake News Dataset, https://github.com/diptamath/covid_fake_news,https://arxiv.org/abs/2011.03327 

import pandas as pd
trainingdata=pd.read_csv("https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv", usecols = ['tweet','label'])
testdata=pd.read_csv("https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/english_test_with_labels.csv", usecols = ['tweet','label'])

trainingdata

,tweet,label
0,The CDC currently reports 99031 deaths. In gen...,real
1,States reported 1121 deaths a small rise from ...,real
2,Politically Correct Woman (Almost) Uses Pandem...,fake
3,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,Populous states can generate large case counts...,real
...,...,...
6415,A tiger tested positive for COVID-19 please st...,fake
6416,???Autopsies prove that COVID-19 is??� a blood...,fake
6417,_A post claims a COVID-19 vaccine has already ...,fake
6418,Aamir Khan Donate 250 Cr. In PM Relief Cares Fund,fake


**************************************

Example of real news twee

In [11]:
trainingdata[trainingdata.label=="real"].loc[0, 'tweet']

'The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly 100000 people today.'

Example of fake news tweet

In [12]:
trainingdata[trainingdata.label=="fake"].loc[2, 'tweet']

'Politically Correct Woman (Almost) Uses Pandemic as Excuse Not to Reuse Plastic Bag https://t.co/thF8GuNFPe #coronavirus #nashville'

## Define Preprocessor

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Build vocabulary from training text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(trainingdata.tweet)

# preprocessor tokenizes words and makes sure all documents have the same length
def preprocessor(data, maxlen, max_words):

    sequences = tokenizer.texts_to_sequences(data)

    word_index = tokenizer.word_index
    X = pad_sequences(sequences, maxlen=maxlen)

    return X

## Prepare Train and Test Data

In [22]:
# tokenize and pad X data
X_train = preprocessor(trainingdata.tweet, maxlen=40, max_words=10000)
X_test = preprocessor(testdata.tweet, maxlen=40, max_words=10000)

# one hot encode Y data
y_train = pd.get_dummies(trainingdata.label)
y_test = pd.get_dummies(testdata.label)

In [48]:
print(X_train.shape)
print(X_test.shape)

(6420, 40)
(2140, 40)


### Import Libraries

In [34]:
from tensorflow.keras.layers import SimpleRNN, LSTM, Embedding, SimpleRNN, Flatten, Dense, Bidirectional, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import layers
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D

### Model Evaluation Function

In [24]:
# performance metrics
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score, mean_squared_error, r2_score, mean_absolute_error
import pandas as pd
from math import sqrt

def model_eval_metrics(y_true, y_pred,classification="TRUE"):
     if classification=="TRUE":
        accuracy_eval = accuracy_score(y_true, y_pred)
        f1_score_eval = f1_score(y_true, y_pred,average="macro",zero_division=0)
        precision_eval = precision_score(y_true, y_pred,average="macro",zero_division=0)
        recall_eval = recall_score(y_true, y_pred,average="macro",zero_division=0)
        mse_eval = 0
        rmse_eval = 0
        mae_eval = 0
        r2_eval = 0
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     else:
        accuracy_eval = 0
        f1_score_eval = 0
        precision_eval = 0
        recall_eval = 0
        mse_eval = mean_squared_error(y_true, y_pred)
        rmse_eval = sqrt(mean_squared_error(y_true, y_pred))
        mae_eval = mean_absolute_error(y_true, y_pred)
        r2_eval = r2_score(y_true, y_pred)
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     return finalmetricdata

In [49]:
# set seed for reproducibility
seed = 99
import tensorflow as tf
tf.random.set_seed(seed)

## Baseline Model 

In [ ]:
tf.random.set_seed(seed)

# replace this model with the architectures from the task description
model = Sequential()
model.add(Embedding(10000, 16, input_length=40))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
161/161 [==============================] - 1s 4ms/step - loss: 0.5854 - acc: 0.7034 - val_loss: 0.3732 - val_acc: 0.8294
Epoch 2/10
161/161 [==============================] - 0s 3ms/step - loss: 0.3232 - acc: 0.8564 - val_loss: 0.2627 - val_acc: 0.8910
Epoch 3/10
161/161 [==============================] - 1s 4ms/step - loss: 0.2007 - acc: 0.9235 - val_loss: 0.2129 - val_acc: 0.9143
Epoch 4/10
161/161 [==============================] - 1s 3ms/step - loss: 0.1455 - acc: 0.9471 - val_loss: 0.1897 - val_acc: 0.9252
Epoch 5/10
161/161 [==============================] - 0s 3ms/step - loss: 0.0983 - acc: 0.9655 - val_loss: 0.1785 - val_acc: 0.9252
Epoch 6/10
161/161 [==============================] - 0s 3ms/step - loss: 0.0753 - acc: 0.9716 - val_loss: 0.1718 - val_acc: 0.9322
Epoch 7/10
161/161 [==============================] - 0s 3ms/step - loss: 0.0559 - acc: 0.9793 - val_loss: 0.1727 - val_acc: 0.9245
Epoch 8/10
161/161 [==============================] - 0s 3ms/step - loss: 0.

In [ ]:
# format y_pred as labels 
y_pred = model.predict(X_test).argmax(axis=1)
predicted_labels = [y_test.columns[i] for i in y_pred]

In [ ]:
# y_test is one hot encoded so we need to extract labels before runing model_eval_metrics()
y_test_labels=y_test.idxmax(axis=1) #extract labels from one hot encoded y_test object
y_test_labels=list(y_test.idxmax(axis=1)) #returns a pandas series of predicted labels

In [ ]:
model_eval_metrics( y_test_labels,predicted_labels,classification="TRUE")

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.942056,0.941929,0.941929,0.941929,0,0,0,0


********************************************

### Sequential Model with Embedding layer

In [ ]:
tf.random.set_seed(seed)

# replace this model with the architectures from the task description
model = Sequential()
model.add(Embedding(10000, 40, input_length=40))
model.add(LSTM(20, activation='sigmoid', return_sequences=True))
model.add(Flatten())
model.add(Dense(2, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

history = model.fit(X_train, y_train,
                    epochs=25,
                    batch_size=64,
                    validation_split=0.2)

Epoch 1/25
81/81 [==============================] - 4s 25ms/step - loss: 0.7006 - acc: 0.5348 - val_loss: 0.5676 - val_acc: 0.7383
Epoch 2/25
81/81 [==============================] - 2s 22ms/step - loss: 0.5189 - acc: 0.7503 - val_loss: 0.4913 - val_acc: 0.7858
Epoch 3/25
81/81 [==============================] - 2s 21ms/step - loss: 0.4117 - acc: 0.8039 - val_loss: 0.4045 - val_acc: 0.8294
Epoch 4/25
81/81 [==============================] - 2s 21ms/step - loss: 0.3138 - acc: 0.8643 - val_loss: 0.3134 - val_acc: 0.8723
Epoch 5/25
81/81 [==============================] - 2s 20ms/step - loss: 0.2026 - acc: 0.9176 - val_loss: 0.2544 - val_acc: 0.9026
Epoch 6/25
81/81 [==============================] - 2s 21ms/step - loss: 0.1524 - acc: 0.9416 - val_loss: 0.2368 - val_acc: 0.9058
Epoch 7/25
81/81 [==============================] - 2s 21ms/step - loss: 0.1171 - acc: 0.9535 - val_loss: 0.2338 - val_acc: 0.9120
Epoch 8/25
81/81 [==============================] - 2s 21ms/step - loss: 0.0945 - a

In [ ]:
# format y_pred as labels 
y_pred = model.predict(X_test).argmax(axis=1)
predicted_labels = [y_test.columns[i] for i in y_pred]

In [ ]:
# y_test is one hot encoded so we need to extract labels before runing model_eval_metrics()
y_test_labels=y_test.idxmax(axis=1) #extract labels from one hot encoded y_test object
y_test_labels=list(y_test.idxmax(axis=1)) #returns a pandas series of predicted labels

In [ ]:
model_eval_metrics( y_test_labels,predicted_labels,classification="TRUE")

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.933645,0.933542,0.93334,0.93385,0,0,0,0


### Glove Embedding layers model

In [ ]:
# Download Glove embedding matrix weights (pretrained embeddings)
! wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2021-04-12 16:52:29--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2021-04-12 16:52:29--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2021-04-12 16:52:29--  http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [applic

In [ ]:
! unzip glove.6B.zip 

Archive:  glove.6B.zip
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        


In [ ]:
# Extract embedding data for 100 feature embedding matrix
import os
glove_dir = os.getcwd()

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400001 word vectors.


In [ ]:
# Build embedding matrix
embedding_dim = 100 # change if you use txt files using larger number of features
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(trainingdata.tweet)
word_index = tokenizer.word_index


embedding_matrix = np.zeros((10000, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < 10000:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [ ]:
# Set up same model architecture as before and then import Glove weights to Embedding layer:
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(10000, embedding_dim, input_length=40))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Flatten())
model.add(Dense(2, activation='sigmoid'))
# Add weights in same manner as transfer learning and turn of trainable option before fitting model to freeze weights.
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

mc = ModelCheckpoint('best_glovemodel.h5', monitor='auc',mode='max', verbose=1, save_best_only=True) # evaluating val_acc maximization
red_lr= ReduceLROnPlateau(monitor='auc',patience=2,verbose=1,factor=0.5, min_lr=0.001)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc','AUC'])
history = model.fit(X_train, y_train,
                    epochs=25,
                    batch_size=12, 
                    callbacks=[mc,red_lr])


# model.summary()

Epoch 1/25
535/535 [==============================] - 2s 2ms/step - loss: 0.4617 - acc: 0.7789 - auc: 0.8580

Epoch 00001: auc improved from -inf to 0.90177, saving model to best_glovemodel.h5
Epoch 2/25
535/535 [==============================] - 1s 2ms/step - loss: 0.2246 - acc: 0.9080 - auc: 0.9711

Epoch 00002: auc improved from 0.90177 to 0.96744, saving model to best_glovemodel.h5
Epoch 3/25
535/535 [==============================] - 1s 2ms/step - loss: 0.1577 - acc: 0.9413 - auc: 0.9867

Epoch 00003: auc improved from 0.96744 to 0.98521, saving model to best_glovemodel.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.001.
Epoch 4/25
535/535 [==============================] - 1s 2ms/step - loss: 0.1013 - acc: 0.9641 - auc: 0.9951

Epoch 00004: auc improved from 0.98521 to 0.99364, saving model to best_glovemodel.h5
Epoch 5/25
535/535 [==============================] - 1s 2ms/step - loss: 0.0641 - acc: 0.9827 - auc: 0.9986

Epoch 00005: auc improved from 0.99364 to 0.

In [ ]:
# format y_pred as labels 
y_pred = model.predict(X_test).argmax(axis=1)
predicted_labels = [y_test.columns[i] for i in y_pred]

In [ ]:
# y_test is one hot encoded so we need to extract labels before runing model_eval_metrics()
y_test_labels=y_test.idxmax(axis=1) #extract labels from one hot encoded y_test object
y_test_labels=list(y_test.idxmax(axis=1)) #returns a pandas series of predicted labels

In [ ]:
model_eval_metrics( y_test_labels,predicted_labels,classification="TRUE")

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.864486,0.864087,0.864431,0.863839,0,0,0,0


### Recurrent Neural Network (RNN) model

In [ ]:
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(10000, 16, input_length=40))
model.add(SimpleRNN(16))
model.add(Dense(2, activation='sigmoid'))

mc = ModelCheckpoint('best_rnnmodel.h5', monitor='auc',mode='max', verbose=1, save_best_only=True) # evaluating val_acc maximization
red_lr= ReduceLROnPlateau(monitor='auc',patience=2,verbose=1,factor=0.5, min_lr=0.001)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc', 'AUC'])
history = model.fit(X_train, y_train,
                    epochs=15,
                    batch_size=32,verbose=1,
                    callbacks=[mc,red_lr])

Epoch 1/15
201/201 [==============================] - 4s 10ms/step - loss: 0.5410 - acc: 0.7490 - auc: 0.8224

Epoch 00001: auc improved from -inf to 0.90291, saving model to best_rnnmodel.h5
Epoch 2/15
201/201 [==============================] - 2s 10ms/step - loss: 0.2340 - acc: 0.9171 - auc: 0.9753

Epoch 00002: auc improved from 0.90291 to 0.97519, saving model to best_rnnmodel.h5
Epoch 3/15
201/201 [==============================] - 2s 9ms/step - loss: 0.1429 - acc: 0.9508 - auc: 0.9896

Epoch 00003: auc improved from 0.97519 to 0.98878, saving model to best_rnnmodel.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.001.
Epoch 4/15
201/201 [==============================] - 2s 9ms/step - loss: 0.0965 - acc: 0.9673 - auc: 0.9944

Epoch 00004: auc improved from 0.98878 to 0.99396, saving model to best_rnnmodel.h5
Epoch 5/15
201/201 [==============================] - 2s 9ms/step - loss: 0.0809 - acc: 0.9731 - auc: 0.9952

Epoch 00005: auc improved from 0.99396 to 0.99569,

In [ ]:
# format y_pred as labels 
y_pred = model.predict(X_test).argmax(axis=1)
predicted_labels = [y_test.columns[i] for i in y_pred]

In [ ]:
# y_test is one hot encoded so we need to extract labels before runing model_eval_metrics()
y_test_labels=y_test.idxmax(axis=1) #extract labels from one hot encoded y_test object
y_test_labels=list(y_test.idxmax(axis=1)) #returns a pandas series of predicted labels

In [ ]:
model_eval_metrics( y_test_labels,predicted_labels,classification="TRUE")

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.915421,0.915041,0.916536,0.914294,0,0,0,0


### Stacked Recurrent Neural Network (RNN) model


In [ ]:
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(10000, 116, input_length=40))
model.add(LSTM(116, return_sequences=True))
model.add(SimpleRNN(64, return_sequences=True))
model.add(SimpleRNN(64, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(2, activation='softmax'))

mc = ModelCheckpoint('best_rnnstackedmodel.h5', monitor='auc',mode='max', verbose=1, save_best_only=True) # evaluating val_acc maximization
red_lr= ReduceLROnPlateau(monitor='AUC',patience=2,verbose=1,factor=0.5, min_lr=0.001)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc','AUC'])
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=32,callbacks=[mc,red_lr])

Epoch 1/20
201/201 [==============================] - 22s 86ms/step - loss: 0.4840 - acc: 0.7705 - auc: 0.8347

Epoch 00001: auc improved from -inf to 0.92301, saving model to best_rnnstackedmodel.h5
Epoch 2/20
201/201 [==============================] - 17s 84ms/step - loss: 0.1384 - acc: 0.9475 - auc: 0.9843

Epoch 00002: auc improved from 0.92301 to 0.98428, saving model to best_rnnstackedmodel.h5
Epoch 3/20
201/201 [==============================] - 17s 83ms/step - loss: 0.0752 - acc: 0.9764 - auc: 0.9923

Epoch 00003: auc improved from 0.98428 to 0.99102, saving model to best_rnnstackedmodel.h5
Epoch 4/20
201/201 [==============================] - 17s 84ms/step - loss: 0.0491 - acc: 0.9861 - auc: 0.9945

Epoch 00004: auc improved from 0.99102 to 0.99421, saving model to best_rnnstackedmodel.h5
Epoch 5/20
201/201 [==============================] - 17s 86ms/step - loss: 0.0355 - acc: 0.9878 - auc: 0.9971

Epoch 00005: auc improved from 0.99421 to 0.99668, saving model to best_rnnstac

In [ ]:
# format y_pred as labels 
y_pred = model.predict(X_test).argmax(axis=1)
predicted_labels = [y_test.columns[i] for i in y_pred]

In [ ]:
# y_test is one hot encoded so we need to extract labels before runing model_eval_metrics()
y_test_labels=y_test.idxmax(axis=1) #extract labels from one hot encoded y_test object
y_test_labels=list(y_test.idxmax(axis=1)) #returns a pandas series of predicted labels

In [ ]:
model_eval_metrics( y_test_labels,predicted_labels,classification="TRUE")

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.940654,0.940556,0.940373,0.940809,0,0,0,0


In [ ]:
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(10000, 116, input_length=40))
model.add(LSTM(116, return_sequences=True))
model.add(SimpleRNN(116, return_sequences=True))
model.add(SimpleRNN(64, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Flatten())
model.add(Dense(2, activation='sigmoid'))

mc = ModelCheckpoint('best_rnnstacked2model.h5', monitor='auc',mode='max', verbose=1, save_best_only=True) # evaluating val_acc maximization
red_lr= ReduceLROnPlateau(monitor='auc',patience=2,verbose=1,factor=0.5, min_lr=0.001)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc','AUC'])
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,callbacks=[mc,red_lr])

Epoch 1/50
201/201 [==============================] - 21s 75ms/step - loss: 0.6043 - acc: 0.6635 - auc: 0.7165

Epoch 00001: auc improved from -inf to 0.85879, saving model to best_rnnstacked2model.h5
Epoch 2/50
201/201 [==============================] - 15s 76ms/step - loss: 0.2296 - acc: 0.9179 - auc: 0.9694

Epoch 00002: auc improved from 0.85879 to 0.97182, saving model to best_rnnstacked2model.h5
Epoch 3/50
201/201 [==============================] - 15s 75ms/step - loss: 0.1229 - acc: 0.9622 - auc: 0.9883

Epoch 00003: auc improved from 0.97182 to 0.98910, saving model to best_rnnstacked2model.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.001.
Epoch 4/50
201/201 [==============================] - 15s 75ms/step - loss: 0.0791 - acc: 0.9782 - auc: 0.9936

Epoch 00004: auc improved from 0.98910 to 0.99174, saving model to best_rnnstacked2model.h5
Epoch 5/50
201/201 [==============================] - 15s 75ms/step - loss: 0.0722 - acc: 0.9765 - auc: 0.9944

Epoch 0000

In [ ]:
# format y_pred as labels 
y_pred = model.predict(X_test).argmax(axis=1)
predicted_labels = [y_test.columns[i] for i in y_pred]

In [ ]:
# y_test is one hot encoded so we need to extract labels before runing model_eval_metrics()
y_test_labels=y_test.idxmax(axis=1) #extract labels from one hot encoded y_test object
y_test_labels=list(y_test.idxmax(axis=1)) #returns a pandas series of predicted labels

In [ ]:
model_eval_metrics( y_test_labels,predicted_labels,classification="TRUE")

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.947664,0.947571,0.947415,0.947768,0,0,0,0


In [ ]:
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(10000, 116, input_length=40))
model.add(LSTM(116, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))
model.add(SimpleRNN(116, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))
model.add(SimpleRNN(64, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))
model.add(SimpleRNN(32, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))
model.add(SimpleRNN(32, activation='sigmoid', dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(32, activation='sigmoid'))
model.add(Flatten())
model.add(Dense(2, activation='sigmoid'))

mc = ModelCheckpoint('best_rnnstacked2model.h5', monitor='auc',mode='max', verbose=1, save_best_only=True) # evaluating val_acc maximization
red_lr= ReduceLROnPlateau(monitor='auc',patience=2,verbose=1,factor=0.5, min_lr=0.001)

# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc','AUC'])
# history = model.fit(X_train, y_train,
#                     epochs=24,
#                     batch_size=32,callbacks=[mc,red_lr])

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc','AUC'])
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,callbacks=[mc,red_lr])

Epoch 1/50
201/201 [==============================] - 41s 167ms/step - loss: 0.6126 - acc: 0.6556 - auc: 0.7049

Epoch 00001: auc improved from -inf to 0.84954, saving model to best_rnnstacked2model.h5
Epoch 2/50
201/201 [==============================] - 34s 167ms/step - loss: 0.2406 - acc: 0.9195 - auc: 0.9656

Epoch 00002: auc improved from 0.84954 to 0.97088, saving model to best_rnnstacked2model.h5
Epoch 3/50
201/201 [==============================] - 34s 168ms/step - loss: 0.1317 - acc: 0.9586 - auc: 0.9883

Epoch 00003: auc improved from 0.97088 to 0.98734, saving model to best_rnnstacked2model.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.001.
Epoch 4/50
201/201 [==============================] - 34s 167ms/step - loss: 0.0867 - acc: 0.9740 - auc: 0.9925

Epoch 00004: auc improved from 0.98734 to 0.99320, saving model to best_rnnstacked2model.h5
Epoch 5/50
201/201 [==============================] - 34s 168ms/step - loss: 0.0722 - acc: 0.9784 - auc: 0.9956

Epoch

In [ ]:
# format y_pred as labels 
y_pred = model.predict(X_test).argmax(axis=1)
predicted_labels = [y_test.columns[i] for i in y_pred]

In [ ]:
# y_test is one hot encoded so we need to extract labels before runing model_eval_metrics()
y_test_labels=y_test.idxmax(axis=1) #extract labels from one hot encoded y_test object
y_test_labels=list(y_test.idxmax(axis=1)) #returns a pandas series of predicted labels

In [ ]:
model_eval_metrics( y_test_labels,predicted_labels,classification="TRUE")

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.942523,0.942474,0.942305,0.943207,0,0,0,0


### Long Short-Term Memory (LSTM) Sequential Model with Embedding layer and Dropout

In [ ]:
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(10000, 64, input_length=40))
model.add(LSTM(164, dropout=0.1, recurrent_dropout=0.1)) 
model.add(Dense(2, activation='sigmoid'))

mc = ModelCheckpoint('best_lstm_model.h5', monitor='auc',mode='max', verbose=1, save_best_only=True) # evaluating val_acc maximization
red_lr= ReduceLROnPlateau(monitor='auc',patience=2,verbose=1,factor=0.05, min_lr=0.001)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc','AUC'])
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=64,
                    validation_split=0.2,callbacks=[mc,red_lr])

Epoch 1/20
81/81 [==============================] - 21s 225ms/step - loss: 0.5551 - acc: 0.7005 - auc: 0.7743 - val_loss: 0.2554 - val_acc: 0.9050 - val_auc: 0.9631

Epoch 00001: auc improved from -inf to 0.88257, saving model to best_lstm_model.h5
Epoch 2/20
81/81 [==============================] - 17s 215ms/step - loss: 0.1587 - acc: 0.9425 - auc: 0.9867 - val_loss: 0.1872 - val_acc: 0.9268 - val_auc: 0.9788

Epoch 00002: auc improved from 0.88257 to 0.98698, saving model to best_lstm_model.h5
Epoch 3/20
81/81 [==============================] - 17s 215ms/step - loss: 0.0763 - acc: 0.9764 - auc: 0.9971 - val_loss: 0.2165 - val_acc: 0.9167 - val_auc: 0.9742

Epoch 00003: auc improved from 0.98698 to 0.99745, saving model to best_lstm_model.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.001.
Epoch 4/20
81/81 [==============================] - 17s 208ms/step - loss: 0.0276 - acc: 0.9920 - auc: 0.9996 - val_loss: 0.2621 - val_acc: 0.9159 - val_auc: 0.9698

Epoch 00004: auc

In [ ]:
# format y_pred as labels 
y_pred = model.predict(X_test).argmax(axis=1)
predicted_labels = [y_test.columns[i] for i in y_pred]

In [ ]:
# y_test is one hot encoded so we need to extract labels before runing model_eval_metrics()
y_test_labels=y_test.idxmax(axis=1) #extract labels from one hot encoded y_test object
y_test_labels=list(y_test.idxmax(axis=1)) #returns a pandas series of predicted labels

In [ ]:
model_eval_metrics( y_test_labels,predicted_labels,classification="TRUE")

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.935047,0.934954,0.934735,0.93532,0,0,0,0


### Bidriectional LSTM model

In [ ]:

model = Sequential()
model.add(Embedding(10000, 116, input_length=40))
model.add(Bidirectional(LSTM(116)))
model.add(Flatten())
model.add(Dense(2, activation='sigmoid'))

mc = ModelCheckpoint('best_lstm_model.h5', monitor='auc',mode='max', verbose=1, save_best_only=True) # evaluating val_acc maximization
red_lr= ReduceLROnPlateau(monitor='auc',patience=2,verbose=1,factor=0.05, min_lr=0.001)

model.compile(optimizer=RMSprop(), loss='binary_crossentropy', metrics=['acc','AUC'])
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=116,
                    validation_split=0.2,callbacks=[mc,red_lr])

Epoch 1/20
45/45 [==============================] - 17s 291ms/step - loss: 0.5138 - acc: 0.7344 - auc: 0.8142 - val_loss: 0.2531 - val_acc: 0.9073 - val_auc: 0.9686

Epoch 00001: auc improved from -inf to 0.90520, saving model to best_lstm_model.h5
Epoch 2/20
45/45 [==============================] - 12s 266ms/step - loss: 0.1502 - acc: 0.9459 - auc: 0.9873 - val_loss: 0.2073 - val_acc: 0.9229 - val_auc: 0.9784

Epoch 00002: auc improved from 0.90520 to 0.98546, saving model to best_lstm_model.h5
Epoch 3/20
45/45 [==============================] - 12s 267ms/step - loss: 0.0759 - acc: 0.9752 - auc: 0.9963 - val_loss: 0.2355 - val_acc: 0.9034 - val_auc: 0.9721

Epoch 00003: auc improved from 0.98546 to 0.99550, saving model to best_lstm_model.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.001.
Epoch 4/20
45/45 [==============================] - 12s 266ms/step - loss: 0.0478 - acc: 0.9838 - auc: 0.9982 - val_loss: 0.3946 - val_acc: 0.8435 - val_auc: 0.9312

Epoch 00004: auc

In [ ]:
# format y_pred as labels 
y_pred = model.predict(X_test).argmax(axis=1)
predicted_labels = [y_test.columns[i] for i in y_pred]

In [ ]:
# y_test is one hot encoded so we need to extract labels before runing model_eval_metrics()
y_test_labels=y_test.idxmax(axis=1) #extract labels from one hot encoded y_test object
y_test_labels=list(y_test.idxmax(axis=1)) #returns a pandas series of predicted labels

In [ ]:
model_eval_metrics( y_test_labels,predicted_labels,classification="TRUE")

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.938318,0.93824,0.938009,0.938708,0,0,0,0


### Stacked Bidirectional LSTM model

In [ ]:
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(10000, 64, input_length=40))
model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2, activation='sigmoid', return_sequences=True)))
model.add(Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.2, activation='sigmoid')))
model.add(Dense(2,activation='sigmoid'))

mc = ModelCheckpoint('best_bilstmstacked_model.h5', monitor='auc',mode='max', verbose=1, save_best_only=True) # evaluating val_acc maximization
red_lr= ReduceLROnPlateau(monitor='auc',patience=2,verbose=1,factor=0.05, min_lr=0.001)

model.compile(loss='binary_crossentropy', optimizer=RMSprop(),  metrics=['acc','AUC'])

history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=64,
                    validation_split=0.2,callbacks=[mc,red_lr])

Epoch 1/20
81/81 [==============================] - 36s 302ms/step - loss: 0.6911 - acc: 0.5378 - auc: 0.5381 - val_loss: 0.6166 - val_acc: 0.6332 - val_auc: 0.7353

Epoch 00001: auc improved from -inf to 0.61413, saving model to best_bilstmstacked_model.h5
Epoch 2/20
81/81 [==============================] - 23s 287ms/step - loss: 0.5310 - acc: 0.7722 - auc: 0.8456 - val_loss: 0.3692 - val_acc: 0.8294 - val_auc: 0.9222

Epoch 00002: auc improved from 0.61413 to 0.88006, saving model to best_bilstmstacked_model.h5
Epoch 3/20
81/81 [==============================] - 23s 288ms/step - loss: 0.2989 - acc: 0.8828 - auc: 0.9512 - val_loss: 0.3998 - val_acc: 0.8326 - val_auc: 0.9190

Epoch 00003: auc improved from 0.88006 to 0.96126, saving model to best_bilstmstacked_model.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.001.
Epoch 4/20
81/81 [==============================] - 23s 288ms/step - loss: 0.1991 - acc: 0.9266 - auc: 0.9781 - val_loss: 0.2264 - val_acc: 0.9136 - val_au

In [ ]:
# format y_pred as labels 
y_pred = model.predict(X_test).argmax(axis=1)
predicted_labels = [y_test.columns[i] for i in y_pred]

In [ ]:
# y_test is one hot encoded so we need to extract labels before runing model_eval_metrics()
y_test_labels=y_test.idxmax(axis=1) #extract labels from one hot encoded y_test object
y_test_labels=list(y_test.idxmax(axis=1)) #returns a pandas series of predicted labels

In [ ]:
model_eval_metrics( y_test_labels,predicted_labels,classification="TRUE")

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.921028,0.920253,0.927184,0.918426,0,0,0,0


### Seqeuntial Model with Emedding and 1D Convnet layers

In [ ]:
tf.random.set_seed(seed)

model = Sequential()
model.add(layers.Embedding(10000, 116, input_length=40))
model.add(layers.Conv1D(64, 1, activation='softmax')) 
model.add(layers.MaxPooling1D(10)) #
model.add(layers.Conv1D(32, 1, activation='relu')) 
model.add(layers.MaxPooling1D(2)) #
model.add(layers.Conv1D(32, 1, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(2))

mc = ModelCheckpoint('best_1d_model.h5', monitor='auc',mode='max', verbose=1, save_best_only=True) # evaluating val_acc maximization
red_lr= ReduceLROnPlateau(monitor='auc',patience=2,verbose=1,factor=0.05, min_lr=0.001)

model.compile(optimizer=RMSprop(), loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=164,
                    validation_split=0.2,callbacks=[mc,red_lr])

Epoch 1/20
32/32 [==============================] - 3s 45ms/step - loss: 1.4596 - acc: 0.5038 - val_loss: 0.8477 - val_acc: 0.7142
Epoch 2/20
32/32 [==============================] - 1s 34ms/step - loss: 0.7839 - acc: 0.6119 - val_loss: 0.6881 - val_acc: 0.5148
Epoch 3/20
32/32 [==============================] - 1s 35ms/step - loss: 0.6822 - acc: 0.5499 - val_loss: 0.6638 - val_acc: 0.5257
Epoch 4/20
32/32 [==============================] - 1s 33ms/step - loss: 0.6384 - acc: 0.6737 - val_loss: 0.5472 - val_acc: 0.8131
Epoch 5/20
32/32 [==============================] - 1s 34ms/step - loss: 0.4905 - acc: 0.8163 - val_loss: 0.3848 - val_acc: 0.8474
Epoch 6/20
32/32 [==============================] - 1s 34ms/step - loss: 0.3471 - acc: 0.8535 - val_loss: 0.3086 - val_acc: 0.8645
Epoch 7/20
32/32 [==============================] - 1s 34ms/step - loss: 0.2610 - acc: 0.8954 - val_loss: 0.2734 - val_acc: 0.8972
Epoch 8/20
32/32 [==============================] - 1s 35ms/step - loss: 0.2122 - a

In [ ]:
# format y_pred as labels 
y_pred = model.predict(X_test).argmax(axis=1)
predicted_labels = [y_test.columns[i] for i in y_pred]

In [ ]:
# y_test is one hot encoded so we need to extract labels before runing model_eval_metrics()
y_test_labels=y_test.idxmax(axis=1) #extract labels from one hot encoded y_test object
y_test_labels=list(y_test.idxmax(axis=1)) #returns a pandas series of predicted labels

In [ ]:
model_eval_metrics( y_test_labels,predicted_labels,classification="TRUE")

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.937383,0.937031,0.939627,0.935933,0,0,0,0


************************************************

### Evaluation

I created several deep learning models to identify misinformation related to the Covid-19 pandemic in tweets. 

These models included:
- A model with an embedding layer and dense layers
- A model using Conv1d Layers
- Models with LSTM layer(s)
- A model with stacked sequential layers
- A model with bidirectional sequential layers 

Nearly all models acheived above .93 for all performance meterics, including accuracy, f1-score, precision, and recall. 

The best performing model was the **Stacked Recurrent Neural Network (RNN) model**, acheiving nearly .95 for all performance metrics.
The model has both an embedding and LSTM layer and 4 RNN layers. The model also utilizes sigmoid activation.

### Testing my top model with my own sample tweets

In [6]:
best_model = ai.aimsonnx.instantiate_model(api_url, version=69) 

In [15]:
my_test_tweets = ["Covid is not a big deal, its just a cold", # misinformation
                  "Masks are stupid. Covid is joke",  # misinformation

                  "#COVID19 vaccines are a safe way to build protection.", # actual tweet from CDC
                  "Based on COVID-NET data in recent weeks, rates of #COVID19 hospitalizations in adults ages 50–64 have risen faster than other age groups in several states.", # actual tweet from CDC
                  "You can't get Covid from vaccination. Not possible." # Twitter verified tweet by doctor             
]

In [16]:
my_test_tweets_processed = preprocessor(my_test_tweets, maxlen=40, max_words=10000)
best_model.predict(my_test_tweets_processed).argmax(axis=1)

array([1, 1, 1, 1, 1])

### References

Shahi, Gautam Kishore, Anne Dirkson, and Tim A. Majchrzak. "An exploratory study of covid-19 misinformation on twitter." Online Social Networks and Media 22 (2021): 100104.

## Submit Placeholder Model

In [1]:
# install aimodelshare library
%%capture
! pip install aimodelshare --upgrade --extra-index-url https://test.pypi.org/simple/ 

In [2]:
import aimodelshare as ai
from aimodelshare.aimsonnx import model_to_onnx

In [ ]:
# save preprocessor
ai.export_preprocessor(preprocessor,"")

In [ ]:
# save model in onnx format
onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("onnx_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

INFO:tensorflow:Assets written to: /tmp/assets


INFO:tensorflow:Assets written to: /tmp/assets


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# set credentials for modeltoapi function 
# make sure you have uploaded your credentials.txt file
from aimodelshare.aws import set_credentials
api_url = "https://wvr23l2z9i.execute-api.us-east-1.amazonaws.com/prod/m"
cred_path = "/content/drive/My Drive/Adv. Machine Learning/Code/Week 12/credentials.txt"

set_credentials(apiurl=api_url,
                credential_file=cred_path, 
                type="submit_model", 
                manual=False)

AI Model Share login credentials set successfully.
AWS credentials set successfully.


In [ ]:
# submit model and predictions to competition
ai.submit_model("onnx_model.onnx",
                api_url,
                prediction_submission=predicted_labels,
                preprocessor="preprocessor.zip")

'Your model has been submitted as model version 69'

In [ ]:
# check leaderboard
data=ai.get_leaderboard(api_url, verbose=3)
ai.leaderboard.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,bidirectional_layers,conv1d_layers,dense_layers,embedding_layers,flatten_layers,globalmaxpooling1d_layers,lstm_layers,maxpooling1d_layers,simplernn_layers,relu_act,sigmoid_act,softmax_act,tanh_act,loss,optimizer,model_config,username,version
0,95.09%,95.09%,95.07%,95.12%,keras,False,True,Sequential,3,161922,nan,nan,1,1,1.000000,nan,nan,nan,nan,nan,nan,1.000000,nan,str,RMSprop,"{'name': 'sequential', 'layers...",hpeters,67
1,95.09%,95.09%,95.07%,95.12%,keras,False,True,Sequential,3,161922,nan,nan,1,1,1.000000,nan,nan,nan,nan,nan,nan,1.000000,nan,str,RMSprop,"{'name': 'sequential', 'layers...",hpeters,66
2,95.00%,94.99%,94.97%,95.02%,keras,False,True,Sequential,5,1081482,1.000000,nan,2,1,nan,nan,1.000000,nan,nan,1.000000,nan,1.000000,1.000000,str,RMSprop,"{'name': 'sequential_29', 'lay...",kagenlim,61
3,94.86%,94.85%,94.84%,94.87%,keras,False,True,Sequential,5,1035746,nan,nan,2,1,nan,nan,2.000000,nan,nan,1.000000,nan,1.000000,2.000000,str,RMSprop,"{'name': 'sequential_3', 'laye...",kagenlim,19
4,94.77%,94.76%,94.74%,94.78%,keras,False,True,Sequential,9,1313030,nan,nan,2,1,1.000000,nan,1.000000,nan,4.000000,nan,3.000000,nan,4.000000,str,RMSprop,"{'name': 'sequential_1', 'laye...",kka2120,69
5,94.58%,94.57%,94.57%,94.57%,keras,False,True,Sequential,5,1070202,nan,nan,2,1,nan,nan,2.000000,nan,nan,1.000000,nan,1.000000,2.000000,str,RMSprop,"{'name': 'sequential_4', 'laye...",kagenlim,60
6,94.49%,94.47%,94.47%,94.48%,keras,False,True,Sequential,3,161282,nan,nan,1,1,1.000000,nan,nan,nan,nan,nan,nan,1.000000,nan,str,RMSprop,"{'name': 'sequential', 'layers...",newusertest,4
7,94.35%,94.34%,94.32%,94.37%,keras,False,True,Sequential,6,148066,nan,2.000000,1,1,1.000000,nan,nan,1.000000,nan,2.000000,nan,1.000000,nan,str,RMSprop,"{'name': 'sequential_72', 'lay...",prajseth,40
8,94.25%,94.24%,94.24%,94.24%,keras,False,True,Sequential,3,98818,nan,nan,1,1,nan,nan,1.000000,nan,nan,nan,nan,1.000000,1.000000,str,RMSprop,"{'name': 'sequential_78', 'lay...",prajseth,41
9,94.21%,94.19%,94.18%,94.21%,keras,False,True,Sequential,3,402690,nan,nan,1,1,nan,nan,1.000000,nan,nan,nan,1.000000,nan,1.000000,str,RMSprop,"{'name': 'sequential_5', 'laye...",xc2303_xc,63


In [18]:
 # model next to mine on the leaderboard
 nextbestmodel = ai.aimsonnx.instantiate_model(api_url, version=19) 
 nextbestmodel.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 100)           1000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 40, 32)            17024     
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                16600     
_________________________________________________________________
dense_2 (Dense)              (None, 40)                2040      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 82        
Total params: 1,035,746
Trainable params: 1,035,746
Non-trainable params: 0
_________________________________________________________________


In [ ]:
 # my model
 bestmodel = ai.aimsonnx.instantiate_model(api_url, version=69) 
 bestmodel.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 116)           1160000   
_________________________________________________________________
lstm (LSTM)                  (None, 40, 116)           108112    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 40, 116)           27028     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 40, 64)            11584     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 40, 32)            3104      
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense (Dense)                (None, 32)               

In [ ]:
# Compare the two models to see differences
ai.aimsonnx.compare_models(api_url, version_list=[19,69]) 

,Model_19_Layer,Model_19_Shape,Model_19_Params,Model_69_Layer,Model_69_Shape,Model_69_Params
0,Embedding,"(None, 40, 100)",1000000.000000,Embedding,"(None, 40, 116)",1160000
1,LSTM,"(None, 40, 32)",17024.000000,LSTM,"(None, 40, 116)",108112
2,LSTM,"(None, 50)",16600.000000,SimpleRNN,"(None, 40, 116)",27028
3,Dense,"(None, 40)",2040.000000,SimpleRNN,"(None, 40, 64)",11584
4,Dense,"(None, 2)",82.000000,SimpleRNN,"(None, 40, 32)",3104
5,nan,nan,nan,SimpleRNN,"(None, 32)",2080
6,nan,nan,nan,Dense,"(None, 32)",1056
7,nan,nan,nan,Flatten,"(None, 32)",0
8,nan,nan,nan,Dense,"(None, 2)",66


My model used RNN layers as well as a Flatten layer, while the other model used two LSTM layers. The two models acheive nearly the same performance.

***********************************************

In [20]:
# Fit the best model from the leader board to training data and 
# evaluate it on test data to complete your report.
top_model = ai.aimsonnx.instantiate_model(api_url, version=1) 
top_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 3202      
Total params: 163,202
Trainable params: 163,202
Non-trainable params: 0
_________________________________________________________________


In [67]:
# need to adjust preprocessing for top model
X_train = preprocessor(trainingdata.tweet, maxlen=100, max_words=10000)
X_test = preprocessor(testdata.tweet, maxlen=100, max_words=10000)

# one hot encode Y data
y_train = pd.get_dummies(trainingdata.label)
y_test = pd.get_dummies(testdata.label)


model = Sequential()
model.add(Embedding(10000, 16, input_length=100))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/20
161/161 [==============================] - 1s 4ms/step - loss: 0.5705 - acc: 0.7083 - val_loss: 0.3559 - val_acc: 0.8380
Epoch 2/20
161/161 [==============================] - 0s 3ms/step - loss: 0.2993 - acc: 0.8722 - val_loss: 0.2426 - val_acc: 0.8995
Epoch 3/20
161/161 [==============================] - 0s 3ms/step - loss: 0.1845 - acc: 0.9327 - val_loss: 0.2003 - val_acc: 0.9245
Epoch 4/20
161/161 [==============================] - 0s 3ms/step - loss: 0.1386 - acc: 0.9470 - val_loss: 0.1810 - val_acc: 0.9307
Epoch 5/20
161/161 [==============================] - 0s 3ms/step - loss: 0.0944 - acc: 0.9661 - val_loss: 0.1732 - val_acc: 0.9291
Epoch 6/20
161/161 [==============================] - 0s 3ms/step - loss: 0.0741 - acc: 0.9745 - val_loss: 0.1673 - val_acc: 0.9322
Epoch 7/20
161/161 [==============================] - 0s 3ms/step - loss: 0.0591 - acc: 0.9770 - val_loss: 0.1678 - val_acc: 0.9291
Epoch 8/20
161/161 [==============================] - 0s 3ms/step - loss: 0.

In [68]:
# format y_pred as labels 
y_pred = model.predict(X_test).argmax(axis=1)
predicted_labels = [y_test.columns[i] for i in y_pred]

In [69]:
# y_test is one hot encoded so we need to extract labels before runing model_eval_metrics()
y_test_labels=y_test.idxmax(axis=1) #extract labels from one hot encoded y_test object
y_test_labels=list(y_test.idxmax(axis=1)) #returns a pandas series of predicted labels

In [70]:
model_eval_metrics( y_test_labels,predicted_labels,classification="TRUE")

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.938785,0.938479,0.940504,0.937535,0,0,0,0


In [71]:
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 100, 16)           160000    
_________________________________________________________________
flatten_20 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 2)                 3202      
Total params: 163,202
Trainable params: 163,202
Non-trainable params: 0
_________________________________________________________________
